# data cleaning for the cashierdetail.csv

In [2]:
import pandas as pd
file_path = r'rawdata\cashierdetail.csv'
# Load CSV file into a pandas DataFrame
df = pd.read_csv(file_path, delimiter=',', encoding='utf-8')

it means the cashierdetail.csv is formatted .
## check data consistance inside the table

In [3]:
# check if every row has same number of columns
with open(file_path, 'r') as f:
    lines = f.readlines()

# Get the number of columns in the header
header_columns = len(lines[0].split(','))
print(lines[0])
print(lines[1])

# Separate the lines into correct and incorrect lines
correct_lines = []
incorrect_lines = []
incorrect_lines.append(lines[0])


# Check each line
for i, line in enumerate(lines):
    num_columns = len(line.split(','))
    if num_columns != header_columns:
        print(f'Row {i} has  {num_columns} of columns instead of {header_columns}')
        incorrect_lines.append(line)
        print(line)
    else:
        correct_lines.append(line)

# Write the correct lines to a new file
correct_file_path = 'format_data/cashierdetail_final.csv'
with open(correct_file_path, 'w') as f:
    # print how many lines are written
    print(f'Writing {len(correct_lines)} line in correct file')
    f.writelines(correct_lines)

# Write the incorrect lines to a new file
incorrect_file_path = 'format_data/cashierdetail-incorrect-lines.csv'
with open(incorrect_file_path, 'w') as f:
    # print how many lines are written
    print(f'Writing {len(incorrect_lines)} line in incorrect file')
    f.writelines(incorrect_lines)



noTrans,ArticleCode,Barcode,sizes,qty,basePrice,salePrice,DiscountType,discountPersen,discountRupiah,DiscExpenses,consignment,consignmentRp,subTotal,payment

01SCS19E010001,BSC.AR-QG0007,00ARQG0093,ALL,1,80000,225000,2,0,125000,1,0,20000,100000,80000

Row 50 has  14 of columns instead of 15
01SCS19E010027,BSC.AR-SC0034,00ARSC0228,S,1,125000,315000,2,0,94500,1,0,44100,220500 176400

Row 10855 has  14 of columns instead of 15
01SCS19E300663,AR.ESB-SE003,20ARSE0247,M 1,204750,315000,2,0,63000,1,0,63000,252000,189000

Writing 11558 line in correct file
Writing 3 line in incorrect file


in this case we can find that this two line missing a "," to isolate different value. we just need manually add one and merge with the correct version.

In [4]:
manual_file_path = r'format_data\cashierdetail-incorrect-lines-manual.csv'

# merage incorrect_file_path and correct_file_path
df1 = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')
df2 = pd.read_csv(manual_file_path, delimiter=',', encoding='utf-8')
df = pd.concat([df1, df2], axis=0)
df.to_csv('format_data/cashierdetail_final.csv', index=False)
# print how many lines in the final file
print(f'Writing {len(df)} line in final file')
df = pd.read_csv('format_data/cashierdetail_final.csv', delimiter=',', encoding='utf-8')

Writing 11559 line in final file


## check depulicant/invalid value

In [2]:
import pandas as pd
correct_file_path = 'format_data/cashierdetail_final.csv'
df = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')

# except the colourName picture notes columns
# df = df.drop(['Notes', 'referenceTrans'], axis=1)
# check if there is null values in the DataFrame
print(df.isnull().values.any())
# print the number of null values in each column
print(df.isnull().sum())

False
noTrans           0
ArticleCode       0
Barcode           0
sizes             0
qty               0
basePrice         0
salePrice         0
DiscountType      0
discountPersen    0
discountRupiah    0
DiscExpenses      0
consignment       0
consignmentRp     0
subTotal          0
payment           0
dtype: int64


In [3]:
import pandas as pd
correct_file_path = 'format_data/cashierdetail_final.csv'
df = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')
print("raw data length: " + str(len(df)))


df = df.drop_duplicates()
length = len(df)
print("drop duplicate data length: " + str(len(df)))

#save to format_data folder
df.to_csv('format_data/cashier_final.csv', index=False)

raw data length: 11558
drop duplicate data length: 11558


In [5]:
import pandas as pd

def show_min_max(df):
    # Iterate over each column in the DataFrame
    for column in df.columns:
        # Check if the column contains numerical values
        if pd.api.types.is_numeric_dtype(df[column]):
            # Find the minimum and maximum values in the column
            min_val = df[column].min()
            max_val = df[column].max()
            # Print the column name with the corresponding min and max values
            print(f'{column}:\n Min value: {min_val}, Max value: {max_val}')

df = pd.read_csv('format_data/cashierdetail_final.csv', delimiter=',', encoding='utf-8')
# Call the function to show the min and max values for each column
show_min_max(df)
print(len(df))

qty:
 Min value: -1, Max value: 2
basePrice:
 Min value: 90, Max value: 1150000
salePrice:
 Min value: 15000, Max value: 1250000
DiscountType:
 Min value: 0, Max value: 3
discountPersen:
 Min value: 0, Max value: 0
discountRupiah:
 Min value: 0, Max value: 324000
DiscExpenses:
 Min value: 0, Max value: 1
consignment:
 Min value: 0, Max value: 0
consignmentRp:
 Min value: 5250, Max value: 187500
subTotal:
 Min value: -111300, Max value: 1250000
payment:
 Min value: -83475, Max value: 1062500
11559


## check duplicate value in noTrans columns

In [6]:
import pandas as pd
correct_file_path = 'format_data/cashierdetail_final.csv'
# Load CSV file into a pandas DataFrame
df = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')

count = df.groupby('noTrans').size()
print('the number of duplicated transactions: ', count[count > 1].count())


grouped = df.groupby(['noTrans', 'Barcode']).size().reset_index(name='count')
filtered = grouped[grouped['count'] > 1]
print('the number of duplicated transactions with same barcode: ', filtered.count())

the number of duplicated transactions:  2380
the number of duplicated transactions with same barcode:  noTrans    0
Barcode    0
count      0
dtype: int64


In [7]:
# select the negative quantity rows
df = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')

# negative quantity rows
negative_qty_rows = df[df['qty'] < 0]
print('\n===================\n')
print('the negative quantity rows\n')
print(negative_qty_rows)





the negative quantity rows

              noTrans   ArticleCode     Barcode sizes  qty  basePrice  \
11307  01SCS19E310345  AR.BPH-RB011  20ARRB0467     L   -1     103350   

       salePrice  DiscountType  discountPersen  discountRupiah  DiscExpenses  \
11307     159000             2               0           47700             1   

       consignment  consignmentRp  subTotal  payment  
11307            0          27825   -111300   -83475  


because the 11307 row has the multiple negative value so it is invaild row. we need disposal this row

In [8]:
import pandas as pd
correct_file_path = 'format_data/cashierdetail_final.csv'
df = pd.read_csv(correct_file_path, delimiter=',', encoding='utf-8')

# Select rows with negative quantity values
negative_qty_rows = df[df['qty'] < 0]

# Delete rows with negative quantity values
df = df.drop(negative_qty_rows.index)

# Save the modified DataFrame to a new CSV file
df.to_csv(correct_file_path, index=False)

print("Modified DataFrame with negative quantity rows removed is saved ")


Modified DataFrame with negative quantity rows removed is saved 


# the cashierdetail.csv has been cleaned